In [1]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)
from remote_api_ggg import remote_api
from datetime import datetime
from dateutil import tz
import pytz
import time
import sys

#path = "/var/www/wb_sanisidro/"
path=''
tokens_users={
    #'SanIsidro23':"c9c7911221d3b371a04dc0d5fea633b31C1F99DD55929821A79B0B469EC7FBF31CDC041C",
    '4TrackDemo':"c9c7911221d3b371a04dc0d5fea633b3C330C2ABDF6EC887B73957D9ED80C67A302674A9"
}

inicio = int(datetime(2024,3,1,0,0,0,tzinfo=tz.gettz('America/Monterrey')).timestamp())
fin = int(datetime(2024,3,31,23,59,59,tzinfo=tz.gettz('America/Monterrey')).timestamp())
data = pd.DataFrame()

In [2]:
for i in tokens_users:
    api = remote_api(tokens_users[i])
    sid, uid = api.autorizacion()
    r = api.obtencion_unidades_ids(sid,'Mazda-CX-50',i,4611686018427387903).json()['items']
    #print(r)
    for i in r:
        try:
            m = api.carga_mensajes(i['id'],sid,inicio,fin).json()

            if m['count'] > 0:
                print(i['id'])
                d = pd.DataFrame(m['messages'])
                pos = pd.DataFrame(list(d['pos']))
                d[['position.latitude','position.longitude']] = pos[['y','x']]

            api.limpiar_mensajes(sid)
            print(f"ok con {i['nm']}")

        except:
            print('problemas con la unidad {}'.format(i['nm']))
        

d.rename(columns={'t':'timestamp','rt':'timestamp registered'},inplace=True)
d = d.sort_values(by='timestamp',ascending=True)
d['diferencia'] = d['timestamp registered'] - d['timestamp']
d['conexión'] = 'Conectado'
d.loc[((d['diferencia']>=2*10) & (d['diferencia']<10*60)),'conexión']='Desconexión 2-10 min'
d.loc[((d['diferencia']>=10*10) & (d['diferencia']<30*60)),'conexión']='Desconexión 10-30 min'
d.loc[((d['diferencia']>=30*10)),'conexión']='Desconexión mas de 30 min'

401331049
ok con Mazda-CX-50


In [4]:
import plotly.express as px
barras = d['conexión'].value_counts().reset_index(drop=False)
fig = px.bar(barras, x='conexión', y='count', color='conexión', text='count',
             labels={"count":'Número de registros'})
fig.show()

In [11]:
import plotly.io as pio
fig = px.scatter_mapbox(d,title='''Conexiones''',
                        lat='position.latitude', lon='position.longitude',color='conexión')
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
#fig.update_geos(lataxis_range=[26,25], lonaxis_range=[-100, -100.5])
#fig.update_traces(marker={"size": 50, "symbol": ["*"]})
pio.write_image(fig, 'mapa.png',height=400,width=1000,scale=6)
fig.show()